In [5]:
import numpy as np
import networkx as nx
import os
from sklearn.cluster import DBSCAN
import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Importa la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire DBSCAN sul grafo
def dbscan_clustering(graph, eps, min_samples):
    """
    Esegue DBSCAN sul grafo per raggruppare i nodi in base alla loro distanza.

    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
        eps (float): La distanza massima tra due punti per essere considerati nel medesimo cluster.
        min_samples (int): Il numero minimo di punti in un vicinato per essere considerati un "core point".

    Returns:
        dict: Un dizionario con l'etichetta di cluster per ciascun nodo.
    """
    # Estrai la matrice di adiacenza come feature
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Trasformiamo la matrice di adiacenza in una matrice di distanze
    distance_matrix = np.max(adjacency_matrix) - adjacency_matrix  # Invertiamo la matrice di adiacenza per ottenere una matrice di distanze
    
    # Applichiamo DBSCAN sulla matrice di distanze
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
    labels = dbscan.fit_predict(distance_matrix)
    
    # Restituisci l'etichetta di cluster per ciascun nodo
    cluster_labels = {node: label for node, label in zip(graph.nodes, labels)}
    
    return cluster_labels

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Parametri per DBSCAN (modificabili)
eps = 0.3  # Distanza massima tra due nodi per essere nel medesimo cluster
min_samples = 5  # Numero minimo di nodi per formare un cluster

# Esegui il clustering con DBSCAN
dbscan_labels = dbscan_clustering(graph, eps, min_samples)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in dbscan_labels.items():
    gr.nodes[node]['cluster'] = label  # Assegniamo l'etichetta del cluster al nodo

# Funzione per sanitizzare attributi e chiavi
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e gli attributi del grafo per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")  # Rimpiazza spazi e trattini
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)
    
    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Applica la sanitizzazione al grafo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_dbscan_clusters.gml"
nx.write_gml(gr, output_path)

print(f"Clustering con DBSCAN completato! Grafo esportato in: {output_path}")

# Visualizza il numero di nodi per ciascun cluster
from collections import Counter
cluster_counts = Counter(dbscan_labels.values())
print(f"Numero di nodi per cluster: {cluster_counts}")


Clustering con DBSCAN completato! Grafo esportato in: graph_with_dbscan_clusters.gml
Numero di nodi per cluster: Counter({-1: 5548, 1: 120, 15: 59, 32: 22, 41: 13, 5: 12, 3: 11, 10: 11, 37: 11, 46: 10, 2: 9, 4: 9, 14: 9, 19: 9, 29: 9, 0: 8, 11: 7, 26: 7, 44: 7, 47: 7, 7: 6, 8: 6, 38: 6, 12: 6, 13: 6, 16: 6, 17: 6, 21: 6, 43: 6, 31: 6, 40: 6, 48: 6, 30: 5, 34: 5, 20: 4, 24: 4, 33: 4, 35: 4, 39: 4, 42: 4, 9: 3, 18: 3, 22: 3, 23: 3, 27: 3, 6: 2, 25: 2, 28: 2, 36: 2, 45: 2})
